In [1]:
import sys
display(sys.path)

['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip',
 '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12',
 '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload',
 '',
 '/Users/onkars/Documents/PSYC681/Problem Set/nlp1_rit_course/myenv/lib/python3.12/site-packages']

In [2]:
from utils.preprocessing import load_data, clean_text
from utils.feature_extraction import extract_all_features
from utils.log_likelihood import (
    count_feature_frequencies,
    calculate_probabilities,
    calculate_log_likelihood,
    rank_features_by_log_likelihood
)
from utils.decision_list import build_decision_list, classify_test_data, get_default_sense, classify_test_instance
from utils.evaluation import evaluate_classifier_with_metrics

ImportError: cannot import name 'classify_test_instance' from 'utils.decision_list' (/Users/onkars/Documents/PSYC681/Problem Set/nlp1_rit_course/section2/utils/decision_list.py)

## Data Ingestion

In [16]:
train_file_bass = '/Users/onkars/Documents/PSYC681/Problem Set/nlp1_rit_course/section2/datasets/bass_trn.txt'
train_file_sake = '/Users/onkars/Documents/PSYC681/Problem Set/nlp1_rit_course/section2/datasets/sake_trn.txt'
test_file_bass = '/Users/onkars/Documents/PSYC681/Problem Set/nlp1_rit_course/section2/datasets/bass_tst.txt'
test_file_sake = '/Users/onkars/Documents/PSYC681/Problem Set/nlp1_rit_course/section2/datasets/sake_tst.txt'

train_data_bass = load_data(train_file_bass)
train_data_sake = load_data(train_file_sake)

test_data_bass = load_data(test_file_bass)
test_data_sake = load_data(test_file_sake)



In [17]:
print(train_data_bass[:3])  
print(train_data_sake[:3])  

[{'sense': 'bass', 'left_context': ['Stephan', 'Weidner,', 'the', 'composer', 'and'], 'target': 'bass', 'right_context': ['player', 'for', 'Boehse', 'Onkelz,', 'a']}, {'sense': 'bass', 'left_context': ['valued', 'at', '$250,000.', 'Another', 'double'], 'target': 'bass', 'right_context': ['trapped', 'in', 'the', 'room', 'is']}, {'sense': '*bass', 'left_context': ['portion', 'of', 'shrimp,', 'mussels,', 'sea'], 'target': 'bass', 'right_context': ['and', 'whatnot', 'in', 'a', 'spicy,']}]
[{'sense': 'sake', 'left_context': ['of', 'the', 'society', 'for', 'the'], 'target': 'sake', 'right_context': ['of', 'their', 're-employment', 'at', 'an']}, {'sense': 'sake', 'left_context': ['sacrifice', 'their', 'souls', 'for', 'the'], 'target': 'sake', 'right_context': ['of', 'their', 'country."', '<DOC', 'id="AFE19960518.0073"']}, {'sense': 'sake', 'left_context': ['however.', 'Making', 'controversy', 'for', "controversy's"], 'target': 'sake', 'right_context': ["isn't", 'what', "he's", 'about.', 'He']

## Feature Extraction

In [18]:
def apply_feature_extraction(data):
    """
    Applies feature extraction to each instance in the dataset.
    :param data: List of dictionaries containing context, target word, and sense.
    :return: List of dictionaries, each containing the extracted features and the sense label.
    """
    feature_data = []
    for instance in data:
        features = extract_all_features(instance)
        features['sense'] = instance['sense']  # Keep the sense label for classification later
        feature_data.append(features)
    
    return feature_data


In [19]:
train_features_bass = apply_feature_extraction(train_data_bass)
test_features_bass = apply_feature_extraction(train_data_bass)

train_features_sake = apply_feature_extraction(train_data_sake)
test_features_sake = apply_feature_extraction(train_data_sake)



In [20]:
# Output the first feature set to inspect
display(train_features_bass[0])  
display(train_features_sake[0])  

{'context_window': ('Stephan',
  'Weidner,',
  'the',
  'composer',
  'and',
  'player',
  'for',
  'Boehse',
  'Onkelz,',
  'a'),
 'word_-1': 'and',
 'word_+1': 'player',
 'word_-2': 'composer',
 'word_+2': 'for',
 'word_pair_-2_-1': ('composer', 'and'),
 'word_pair_-1_+1': ('and', 'player'),
 'word_pair_+1_+2': ('player', 'for'),
 'bag_of_words': ('Boehse',
  'Onkelz,',
  'Stephan',
  'Weidner,',
  'a',
  'and',
  'composer',
  'for',
  'player',
  'the'),
 'sense': 'bass'}

{'context_window': ('of',
  'the',
  'society',
  'for',
  'the',
  'of',
  'their',
  're-employment',
  'at',
  'an'),
 'word_-1': 'the',
 'word_+1': 'of',
 'word_-2': 'for',
 'word_+2': 'their',
 'word_pair_-2_-1': ('for', 'the'),
 'word_pair_-1_+1': ('the', 'of'),
 'word_pair_+1_+2': ('of', 'their'),
 'bag_of_words': ('an',
  'at',
  'for',
  'of',
  'of',
  're-employment',
  'society',
  'the',
  'the',
  'their'),
 'sense': 'sake'}

## Log-Likelihood Calculation

In [21]:
# Count feature frequencies for both senses (bass = fish/music, sake = beer/cause)
freq_bass_fish, freq_bass_music, total_bass_fish, total_bass_music = count_feature_frequencies(
    train_features_bass, sense_label1="*bass", sense_label2="bass"
)

freq_sake_beer, freq_sake_cause, total_sake_beer, total_sake_cause = count_feature_frequencies(
    train_features_sake, sense_label1="*sake", sense_label2="sake"
)



Counting features for sense: *bass and bass
Sense: bass, Feature: context_window, Value: ('Stephan', 'Weidner,', 'the', 'composer', 'and', 'player', 'for', 'Boehse', 'Onkelz,', 'a'), Count: 1
Sense: bass, Feature: word_-1, Value: and, Count: 1
Sense: bass, Feature: word_+1, Value: player, Count: 1
Sense: bass, Feature: word_-2, Value: composer, Count: 1
Sense: bass, Feature: word_+2, Value: for, Count: 1
Sense: bass, Feature: word_pair_-2_-1, Value: ('composer', 'and'), Count: 1
Sense: bass, Feature: word_pair_-1_+1, Value: ('and', 'player'), Count: 1
Sense: bass, Feature: word_pair_+1_+2, Value: ('player', 'for'), Count: 1
Sense: bass, Feature: bag_of_words, Value: ('Boehse', 'Onkelz,', 'Stephan', 'Weidner,', 'a', 'and', 'composer', 'for', 'player', 'the'), Count: 1
Sense: bass, Feature: context_window, Value: ('valued', 'at', '$250,000.', 'Another', 'double', 'trapped', 'in', 'the', 'room', 'is'), Count: 1
Sense: bass, Feature: word_-1, Value: double, Count: 1
Sense: bass, Feature: w

In [22]:
# Calculate probabilities (apply Laplace smoothing)
num_unique_features = len(set([f for data in train_features_bass for f in data if f != 'sense']))  # Unique features

prob_bass_fish = calculate_probabilities(freq_bass_fish, total_bass_fish, num_unique_features=num_unique_features)
prob_bass_music = calculate_probabilities(freq_bass_music, total_bass_music, num_unique_features=num_unique_features)

prob_sake_beer = calculate_probabilities(freq_sake_beer, total_sake_beer, num_unique_features=num_unique_features)
prob_sake_cause = calculate_probabilities(freq_sake_cause, total_sake_cause, num_unique_features=num_unique_features)



In [23]:
# Calculate log-likelihood ratios
log_likelihood_bass = calculate_log_likelihood(prob_bass_fish, prob_bass_music)
log_likelihood_sake = calculate_log_likelihood(prob_sake_beer, prob_sake_cause)



In [24]:
# Rank features by log-likelihood (top features are most predictive)
ranked_bass_features = rank_features_by_log_likelihood(log_likelihood_bass)
ranked_sake_features = rank_features_by_log_likelihood(log_likelihood_sake)

# Output the top 10 features
print("Top 10 features for 'bass':")
for feature, score in ranked_bass_features[:10]:
    print(f"{feature}: {score}")
    
print("\nTop 10 features for 'sake':")
for feature, score in ranked_sake_features[:10]:
    print(f"{feature}: {score}")

Top 10 features for 'bass':
('word_-1', 'sea'): 11.636120848264088
('word_-1', 'largemouth'): 10.987425430274978
('word_pair_-1_+1', ('striped', 'and')): 10.490988543961086
('word_pair_-1_+1', ('sea', 'and')): 10.336837864133829
('word_+1', 'are'): 10.336837864133829
('word_-2', 'Chilean'): 10.336837864133829
('word_pair_-2_-1', ('Chilean', 'sea')): 10.336837864133829
('word_+1', 'fishing.'): 10.249826487144198
('word_-1', 'black'): 10.049155791682047
('word_pair_-2_-1', ('the', 'striped')): 10.049155791682047

Top 10 features for 'sake':
('word_-1', 'with'): 11.330603908176274
('word_-1', 'a'): 11.330603908176274
('word_pair_-1_+1', ('the', 'and')): 11.042921835724492
('word_-1', 'Japanese'): 11.042921835724492
('word_pair_-1_+1', ('with', 'and')): 11.042921835724492
('word_+2', 'in'): 11.042921835724492
('word_-1', 'tablespoons'): 11.042921835724492
('word_-1', 'of'): 11.042921835724492
('context_window', ('and', 'coarsely', 'chopped', '1', 'cup', '1', 'cup', 'chicken', 'stock', '1')

## Decision List Classifier

In [26]:
# Build decision lists for 'bass' and 'sake'
decision_list_bass = build_decision_list(ranked_bass_features, sense_label1="*bass", sense_label2="bass")
decision_list_sake = build_decision_list(ranked_sake_features, sense_label1="*sake", sense_label2="sake")

# Get default senses from training data
default_sense_bass = get_default_sense(train_data_bass, sense_label1="*bass", sense_label2="bass")
default_sense_sake = get_default_sense(train_data_sake, sense_label1="*sake", sense_label2="sake")

# Classify the test data
predictions_bass = classify_test_data(test_data_bass, decision_list_bass, default_sense_bass)
predictions_sake = classify_test_data(test_data_sake, decision_list_sake, default_sense_sake)

KeyError: 'features'

In [38]:
# Get ground truth labels from test data
true_labels_bass = [instance['sense'] for instance in test_data_bass]
true_labels_sake = [instance['sense'] for instance in test_data_sake]



In [39]:
# Evaluate the performance of the 'bass' classifier
accuracy_bass, conf_matrix_bass, precision_bass, recall_bass, f1_bass = evaluate_classifier_with_metrics(predictions_bass, true_labels_bass)

# Evaluate the performance of the 'sake' classifier
accuracy_sake, conf_matrix_sake, precision_sake, recall_sake, f1_sake = evaluate_classifier_with_metrics(predictions_sake, true_labels_sake)

/Users/onkars/Documents/PSYC681/Problem Set/nlp1_rit_course/myenv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/onkars/Documents/PSYC681/Problem Set/nlp1_rit_course/myenv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [40]:
# Output results for 'bass'
print(f"Bass Classifier - Accuracy: {accuracy_bass * 100:.2f}%, Precision: {precision_bass:.2f}, Recall: {recall_bass:.2f}, F1-Score: {f1_bass:.2f}")
print("Bass Confusion Matrix:")
print(conf_matrix_bass)

# Output results for 'sake'
print(f"Sake Classifier - Accuracy: {accuracy_sake * 100:.2f}%, Precision: {precision_sake:.2f}, Recall: {recall_sake:.2f}, F1-Score: {f1_sake:.2f}")
print("Sake Confusion Matrix:")
print(conf_matrix_sake)

Bass Classifier - Accuracy: 56.00%, Precision: 0.28, Recall: 0.50, F1-Score: 0.36
Bass Confusion Matrix:
[[56  0]
 [44  0]]
Sake Classifier - Accuracy: 94.00%, Precision: 0.47, Recall: 0.50, F1-Score: 0.48
Sake Confusion Matrix:
[[ 0  6]
 [ 0 94]]


In [ ]:
# # Calculate baseline accuracy
# baseline_accuracy_bass = calculate_baseline_accuracy(test_data_bass, most_frequent_sense_bass)
# baseline_accuracy_sake = calculate_baseline_accuracy(test_data_sake, most_frequent_sense_sake)

# # Calculate model accuracy using the decision list
# model_accuracy_bass = calculate_model_accuracy(test_data_bass, decision_list_bass, default_sense_bass)
# model_accuracy_sake = calculate_model_accuracy(test_data_sake, decision_list_sake, default_sense_sake)

# # Print out the results
# print(f"Baseline Accuracy for BASS: {baseline_accuracy_bass:.2f}%")
# print(f"Model Accuracy for BASS: {model_accuracy_bass:.2f}%")

# print(f"Baseline Accuracy for SAKE: {baseline_accuracy_sake:.2f}%")
# print(f"Model Accuracy for SAKE: {model_accuracy_sake:.2f}%")

In [42]:
# from collections import Counter

# def get_most_frequent_sense(train_data):
#     """
#     Calculate the most frequent sense from the training data.
    
#     :param train_data: List of dictionaries, where each dictionary contains 'sense' key.
#     :return: The most frequent sense in the training data.
#     """
#     # Extract all the senses from the training data
#     senses = [instance['sense'] for instance in train_data]
    
#     # Count the occurrences of each sense
#     sense_counter = Counter(senses)
    
#     # Return the most common sense (most frequent)
#     most_frequent_sense = sense_counter.most_common(1)[0][0]
    
#     return most_frequent_sense

In [43]:
# # Get the most frequent sense for bass and sake
# most_frequent_sense_bass = get_most_frequent_sense(train_data_bass)
# most_frequent_sense_sake = get_most_frequent_sense(train_data_sake)

# # Print the most frequent senses
# print(f"Most Frequent Sense for BASS: {most_frequent_sense_bass}")
# print(f"Most Frequent Sense for SAKE: {most_frequent_sense_sake}")

Most Frequent Sense for BASS: bass
Most Frequent Sense for SAKE: sake


In [44]:
# # Importing necessary functions from utils
# from utils.evaluation import calculate_baseline_accuracy, calculate_model_accuracy


# # Calculate baseline accuracy
# baseline_accuracy_bass = calculate_baseline_accuracy(test_data_bass, most_frequent_sense_bass)
# baseline_accuracy_sake = calculate_baseline_accuracy(test_data_sake, most_frequent_sense_sake)

# # Calculate model accuracy
# model_accuracy_bass = calculate_model_accuracy(test_data_bass, decision_list_bass, default_sense_bass)
# model_accuracy_sake = calculate_model_accuracy(test_data_sake, decision_list_sake, default_sense_sake)

# # Print results for baseline and model accuracy
# print(f"Baseline Accuracy for BASS: {baseline_accuracy_bass:.2f}%")
# print(f"Model Accuracy for BASS: {model_accuracy_bass:.2f}%")

# print(f"Baseline Accuracy for SAKE: {baseline_accuracy_sake:.2f}%")
# print(f"Model Accuracy for SAKE: {model_accuracy_sake:.2f}%")

ValueError: too many values to unpack (expected 2)